In [ ]:
# ------------------------------------------------------------
# 1. Import libraries
# ------------------------------------------------------------
import pandas as pd

# ------------------------------------------------------------
# 2. Load data (use relative path & CSV file)
# ------------------------------------------------------------
# Make sure your CSV file is located in a subfolder called "data"
# (create it if necessary and place CS_CDS.csv inside)
FILE_DATA = "data/CS_CDS.csv"

# Read the CSV file
turns = pd.read_csv(FILE_DATA)

# ------------------------------------------------------------
# 3. Display only the beginning and end of the DataFrame
# ------------------------------------------------------------
print("Data preview (first 6 rows):")
print(turns.head(6))
print("\nData preview (last 6 rows):")
print(turns.tail(6))


In [ ]:
# ------------------------------------------------------------
# 4. Number of dialogues by age
# ------------------------------------------------------------
# Check that required columns exist
required_cols = {"Age", "Dialogue/Interaction_unit"}
if not required_cols.issubset(turns.columns):
    raise ValueError(f"Missing one of the required columns: {required_cols}")

# Define the ages you want to inspect
ages_to_check = [6, 12, 18]

print("Maximum number of dialogues per age:")
for age in ages_to_check:
    max_dialogue = turns.loc[turns["Age"] == age, "Dialogue/Interaction_unit"].max()
    print(f" Age {age} months → {max_dialogue}")

In [ ]:
# ------------------------------------------------------------
# 5. Average number of turns within dialogues by age
# ------------------------------------------------------------
import statistics

# Ensure required columns exist
required_cols = {"Age", "Dialogue/Interaction_unit", "Number"}
if not required_cols.issubset(turns.columns):
    raise ValueError(f"Missing one of the required columns: {required_cols}")

ages_to_check = [6, 12, 18]

print("Average number of turns per dialogue (by age):")
for age in ages_to_check:
    subset = turns[turns["Age"] == age]
    if subset.empty:
        print(f" Age {age} months → no data available")
        continue
    
    # Group by dialogue unit and get max turns per dialogue, then average across dialogues
    avg_turns = statistics.mean(subset.groupby("Dialogue/Interaction_unit")["Number"].max())
    print(f" Age {age} months → {avg_turns:.2f}")


In [ ]:
# ------------------------------------------------------------
# 6. Distribution of languages by age
# ------------------------------------------------------------
# Check that 'Language' and 'Age' columns exist
required_cols = {"Language", "Age"}
if not required_cols.issubset(turns.columns):
    raise ValueError(f"Missing one of the required columns: {required_cols}")

ages_to_check = [6, 12, 18]

print("Language distributions by age:")
for age in ages_to_check:
    subset = turns[turns["Age"] == age]
    if subset.empty:
        print(f"\nAge {age} months → no data available")
        continue
    print(f"\nAge {age} months:")
    print(subset["Language"].value_counts(normalize=False))  # set True for proportions


In [ ]:
# ------------------------------------------------------------
# 7. Distribution of utterance types by age
# ------------------------------------------------------------

# Check required columns
required_cols = {"Utterance_type", "Age"}
if not required_cols.issubset(turns.columns):
    raise ValueError(f"Missing one of the required columns: {required_cols}")

ages_to_check = [6, 12, 18]

print("Utterance type distributions by age:")
for age in ages_to_check:
    subset = turns[turns["Age"] == age]
    if subset.empty:
        print(f"\nAge {age} months → no data available")
        continue
    print(f"\nAge {age} months:")
    print(subset["Utterance_type"].value_counts(normalize=False))  # use normalize=True for proportions


In [ ]:
# ------------------------------------------------------------
# 8. Distribution of participants by age
# ------------------------------------------------------------

# Check that necessary columns exist
required_cols = {"Participant", "Age"}
if not required_cols.issubset(turns.columns):
    raise ValueError(f"Missing one of the required columns: {required_cols}")

ages_to_check = [6, 12, 18]

print("Participant distributions by age:")
for age in ages_to_check:
    subset = turns[turns["Age"] == age]
    if subset.empty:
        print(f"\nAge {age} months → no data available")
        continue
    print(f"\nAge {age} months:")
    print(subset["Participant"].value_counts(normalize=False))  # normalize=True for proportions


In [ ]:
# ------------------------------------------------------------
# 9. Turn initiations by child vs. CDS, per age
# ------------------------------------------------------------

import numpy as np

required_cols = {"Age", "Participant", "Initiation", "CDS"}
if not required_cols.issubset(turns.columns):
    raise ValueError(f"Missing one of the required columns: {required_cols}")

ages_to_check = [6, 12, 18]

for age in ages_to_check:
    subset = turns[turns["Age"] == age]
    if subset.empty:
        print(f"Age {age} months → no data available")
        continue

    # child-initiated turns (participant == 'TCH')
    init_cs = len(subset[(subset["Participant"] == "TCH") & (subset["Initiation"] == 1)])

    # caregiver-initiated turns (CDS row + initiation)
    init_cds = len(subset[(subset["CDS"] == 1) & (subset["Initiation"] == 1)])

    # use total number of utterances at that age as denominator
    total_utts = len(subset)

    share_cs = init_cs / total_utts if total_utts else np.nan
    share_cds = init_cds / total_utts if total_utts else np.nan

    print(f"\nAge {age} months:")
    print(f"  Child-initiated (TCH): {init_cs} / {total_utts} = {share_cs:.3f}")
    print(f"  CDS-initiated:        {init_cds} / {total_utts} = {share_cds:.3f}")


In [ ]:
import pandas as pd
from pathlib import Path

# ------------------------------------------------------------
# 1. Load data using a relative path
# ------------------------------------------------------------

# define relative path (from current notebook location)
file_path = Path("../data/Final.csv")  # adjust "../" if needed

# Read CSV file
df = pd.read_csv(file_path)

print(f"\n✅ Loaded {len(df):,} rows from {file_path}")

# ------------------------------------------------------------
# 2. Data cleaning
# ------------------------------------------------------------

# Replace missing values in 'Duration' with 0
df["Duration"] = pd.to_numeric(df["Duration"], errors="coerce").fillna(0)

# Drop empty rows
df = df.dropna(how="all")

# ------------------------------------------------------------
# 3. Descriptive summaries
# ------------------------------------------------------------

# 3.1 Total duration by speaker
if "Speaker" in df.columns:
    duration_by_speaker = df.groupby("Speaker")["Duration"].sum().sort_values(ascending=False)
    print("\n🎙️ Total Duration by Speaker (s):")
    print(duration_by_speaker.head(10))  # show top 10
    print("...")
else:
    print("\n⚠️ Column 'Speaker' not found.")

# 3.2 Count of utterances by classification
if "Classification" in df.columns:
    utterance_count = df["Classification"].value_counts()
    print("\n💬 Utterance Count by Classification:")
    print(utterance_count)
else:
    print("\n⚠️ Column 'Classification' not found.")

# 3.3 Average duration by category
if "Category" in df.columns:
    avg_duration_category = df.groupby("Category")["Duration"].mean().sort_values(ascending=False)
    print("\n📊 Average Duration by Category (s):")
    print(avg_duration_category)
else:
    print("\n⚠️ Column 'Category' not found.")

# ------------------------------------------------------------
# 4. Save cleaned version (optional)
# ------------------------------------------------------------
out_path = Path("../data/Final_cleaned.csv")
df.to_csv(out_path, index=False)
print(f"\n✅ Cleaned data saved to: {out_path}")


In [ ]:
# ------------------------------------------------------------
# 9. Proportion of 'initiation' turns between 'tch' and 'cds'
# ------------------------------------------------------------

# Filter only initiation turns
if "Turn" not in df.columns or "Category" not in df.columns:
    raise KeyError("Missing required columns: 'Turn' and/or 'Category'")

initiation_df = df[df["Turn"].str.lower() == "initiation"]

# Count how many initiations were produced by each category
initiation_counts = initiation_df["Category"].value_counts()

# Extract counts safely (0 if not found)
tch_count = initiation_counts.get("tch", 0)
cds_count = initiation_counts.get("cds", 0)
total = tch_count + cds_count

# Calculate and display percentages
if total > 0:
    tch_percent = tch_count / total * 100
    cds_percent = cds_count / total * 100

    print(f"\n🎯 Dialogue Initiation by Category (n = {total}):")
    print(f"  tch: {tch_count:>4}  ({tch_percent:5.1f}%)")
    print(f"  cds: {cds_count:>4}  ({cds_percent:5.1f}%)")
else:
    print("\n⚠️ No initiation data found for 'tch' or 'cds'.")


In [ ]:
import pandas as pd
from pathlib import Path

# ------------------------------------------------------------
# 1. Load data (relative CSV path instead of absolute XLSX)
# ------------------------------------------------------------
data_path = Path("../data/CDS_data.csv")   # Adjust folder as needed
df = pd.read_csv(data_path)

# ------------------------------------------------------------
# 2. Filter: keep only CDS rows, and split dialogue vs non-dialogue
# ------------------------------------------------------------
cds_df = df[df["Category"].str.lower().eq("cds")].copy()

dialogue_df = cds_df[cds_df["Turn"].isin(["initiation", "contingent"])].copy()
non_dialogue_df = cds_df[cds_df["Turn"].str.lower().eq("non-contingent")].copy()

# ------------------------------------------------------------
# 3. Total CDS utterances = denominator for percentages
# ------------------------------------------------------------
total_utterances = cds_df.shape[0]
if total_utterances == 0:
    raise ValueError("cds_df is empty — cannot compute percentages of total utterances.")

# ------------------------------------------------------------
# 4. Clean up classification strings
# ------------------------------------------------------------
for _df in (dialogue_df, non_dialogue_df):
    if "Classification" in _df.columns:
        _df["Classification"] = _df["Classification"].astype(str).str.strip()
    else:
        raise KeyError("Missing column 'Classification' in dataset.")

# ------------------------------------------------------------
# 5. Compute absolute counts
# ------------------------------------------------------------
dialogue_counts = dialogue_df["Classification"].value_counts()
non_dialogue_counts = non_dialogue_df["Classification"].value_counts()

# ------------------------------------------------------------
# 6. Convert counts to % of total CDS utterances
# ------------------------------------------------------------
dialogue_pct_total = (dialogue_counts / total_utterances * 100).round(2)
non_dialogue_pct_total = (non_dialogue_counts / total_utterances * 100).round(2)

# ------------------------------------------------------------
# 7. Combine into a single tidy DataFrame
# ------------------------------------------------------------
all_classes = dialogue_pct_total.index.union(non_dialogue_pct_total.index)

percent_total_df = pd.DataFrame({
    "Dialogue % (of total)": dialogue_pct_total.reindex(all_classes, fill_value=0),
    "Non-Contingent % (of total)": non_dialogue_pct_total.reindex(all_classes, fill_value=0)
})

percent_total_df["% Difference (Dialogue - Non)"] = (
    percent_total_df["Dialogue % (of total)"] -
    percent_total_df["Non-Contingent % (of total)"]
).round(2)

# ------------------------------------------------------------
# 8. Show short, GitHub-friendly output (no full table)
# ------------------------------------------------------------
print("\n📊 Classification percentages relative to total CDS utterances:")
display(percent_total_df.head(6))
display(percent_total_df.tail(6))

# ------------------------------------------------------------
# 9. Optional: save the summary
# ------------------------------------------------------------
out_path = Path("../results/classification_percentages.csv")
out_path.parent.mkdir(exist_ok=True)
percent_total_df.to_csv(out_path)
print(f"\n✅ Summary saved to: {out_path}")


In [ ]:
import pandas as pd

# 1) basic counts -------------------------------------------------
total_utterances = len(df)
tch_mask = df["Speaker"].astype(str).str.lower().eq("tch")
tch_utterances = df[tch_mask].shape[0]

tch_percentage = (tch_utterances / total_utterances * 100) if total_utterances else 0

print(f"Total utterances: {total_utterances}")
print(f"Utterances by 'tch': {tch_utterances} ({tch_percentage:.2f}%)")

# 2) define dialogue vs non-contingent ---------------------------
# make sure 'Turn' exists and is string
if "Turn" not in df.columns:
    raise KeyError("Column 'Turn' not found in dataframe.")

df["Turn"] = df["Turn"].astype(str).str.lower()

df["Dialogue_Status"] = df["Turn"].apply(
    lambda x: "Dialogue" if x in ["initiation", "contingent"] else "Non-Contingent"
)

# 3) define speaker group (tch vs. everyone else) ----------------
df["Speaker"] = df["Speaker"].astype(str)
df["Speaker_Group"] = df["Speaker"].str.lower().apply(
    lambda x: "tch" if x == "tch" else "non-tch"
)

# 4) cross-tab: speaker group × dialogue status ------------------
speaker_dialogue_counts = (
    df.groupby(["Speaker_Group", "Dialogue_Status"])
      .size()
      .unstack(fill_value=0)
)

# add row totals
speaker_dialogue_counts["Total"] = speaker_dialogue_counts.sum(axis=1)

print("\nUtterance counts by speaker group and dialogue status:")
print(speaker_dialogue_counts.head())


In [ ]:
import pandas as pd

# Example: df['Turn'] = ['initiation', 'contingent', 'contingent', 'non-contingent', 'initiation', 'contingent', ...]

# Step 1: Initialize variables
dialogical_units = []  # stores list of dialogical unit lengths
current_unit = 0
inside_unit = False

# Step 2: Iterate through the Turn column
for turn in df['Turn']:
    if turn == 'initiation':
        if inside_unit:
            dialogical_units.append(current_unit)
        current_unit = 1
        inside_unit = True
    elif inside_unit and turn == 'contingent':
        current_unit += 1
    elif inside_unit and turn == 'non-contingent':
        dialogical_units.append(current_unit)
        current_unit = 0
        inside_unit = False
    else:
        continue  # Skip other cases if needed

# Handle final dialogical unit (if it ends the file without closing)
if inside_unit and current_unit > 0:
    dialogical_units.append(current_unit)

# Step 3: Calculate average
if dialogical_units:
    average_turns = sum(dialogical_units) / len(dialogical_units)
    print(f"Average number of turns per dialogical unit: {average_turns:.2f}")
else:
    print("No dialogical units found.")


In [ ]:
# Fisher's Exact Test
import pandas as pd
from scipy.stats import fisher_exact

# assume cds_df already exists and is filtered to Category == 'cds'
df = cds_df.copy()

# 1) clean strings ------------------------------------------------
df["Classification"] = df["Classification"].astype(str).str.strip().str.lower()
df["Turn"] = df["Turn"].astype(str).str.lower()

# 2) merge fine-grained codes into broader ones -------------------
merge_map = {
    "vocal imitation": "imitation",
    "iconic imitation": "imitation",
    "prosodic imitation": "imitation",
    "semantic expansion": "expansion",
    "syntactic expansion": "expansion",
    "repair 1": "repair",
    "repair 2": "repair",
    "repair 3": "repair",
}
df["Classification"] = df["Classification"].replace(merge_map)

# 3) define "in turn" vs. "non-contingent" ------------------------
df["is_in_turn"] = df["Turn"].isin(["contingent", "initiation"])

results = []

# 4) loop over utterance classes ----------------------------------
for cls in df["Classification"].dropna().unique():
    is_cls = df["Classification"] == cls
    is_other = ~is_cls

    a = (is_cls & df["is_in_turn"]).sum()          # this class, in turn
    b = (is_cls & ~df["is_in_turn"]).sum()         # this class, out of turn
    c = (is_other & df["is_in_turn"]).sum()        # all other classes, in turn
    d = (is_other & ~df["is_in_turn"]).sum()       # all other classes, out of turn

    table = [[a, b],
             [c, d]]

    # fisher_exact needs a valid 2x2 with nonnegative ints
    if (a + b > 0) and (c + d > 0):
        oddsratio, p = fisher_exact(table)
    else:
        oddsratio, p = float("nan"), float("nan")

    results.append({
        "utterance_type": cls,
        "in_turn": a,
        "non_contingent": b,
        "odds_ratio": oddsratio,
        "p_value": p,
    })

results_df = pd.DataFrame(results).sort_values("p_value")

# show only the interesting ones, as per github comments
sig_df = results_df[results_df["p_value"] < 0.05]

print("\nSignificant associations (Fisher’s exact, p < .05):")
print(sig_df.head(10))         # don’t dump the whole thing
print("\nAll results (head):")
print(results_df.head(10))


In [ ]:
# Bayesian Comparison of Two Proportions

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta

df = cds_df.copy()
df['Classification'] = df['Classification'].str.strip()
# Merge utterance types into broader categories
df['Classification'] = df['Classification'].replace({
    'vocal imitation': 'imitation',
    'iconic imitation': 'imitation',
    'prosodic imitation': 'imitation',
    'semantic expansion': 'expansion',
    'syntactic expansion': 'expansion',
    'repair 1': 'repair',
    'repair 2': 'repair',
    'repair 3': 'repair'
})

# Create binary column for turn-taking
df['is_in_turn'] = df['Turn'].isin(['contingent', 'initiation'])

# Define the classification of interest
target_class = 'Imperative'  # ← change this to whatever category you're analyzing

# Filter for each group
in_turn = df[df['is_in_turn'] == 'True']
non_contingent = df[df['is_in_turn'] == 'False']

# Count how many imperatives in each group (successes)
k1 = (in_turn['Classification'] == target_class).sum()
k2 = (non_contingent['Classification'] == target_class).sum()

# Total utterances in each group (sample size)
n1 = len(in_turn)
n2 = len(non_contingent)

# Posterior distributions assuming uniform Beta(1,1) priors
posterior1 = beta(a=1 + k1, b=1 + n1 - k1)
posterior2 = beta(a=1 + k2, b=1 + n2 - k2)

# Sample from posteriors
samples1 = posterior1.rvs(10000)
samples2 = posterior2.rvs(10000)
diff_samples = samples1 - samples2

# Summary statistics
mean_diff = np.mean(diff_samples)
cred_interval = np.percentile(diff_samples, [2.5, 97.5])
prob_diff_gt_0 = np.mean(diff_samples > 0)

print(f"Mean difference: {mean_diff:.3f}")
print(f"95% credible interval: {cred_interval}")
print(f"Probability In-Turn > Non-Contingent: {prob_diff_gt_0:.3f}")


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import beta

# work on a copy
df = cds_df.copy()

# 1) clean text columns -------------------------------------------------
df["Classification"] = (
    df["Classification"]
    .astype(str)
    .str.strip()
    .str.lower()
)
df["Turn"] = df["Turn"].astype(str).str.strip().str.lower()

# 2) merge fine-grained labels -----------------------------------------
merge_map = {
    "vocal imitation": "imitation",
    "iconic imitation": "imitation",
    "prosodic imitation": "imitation",
    "semantic expansion": "expansion",
    "syntactic expansion": "expansion",
    "repair 1": "repair",
    "repair 2": "repair",
    "repair 3": "repair",
}
df["Classification"] = df["Classification"].replace(merge_map)

# 3) define turn-taking vs. non-contingent ------------------------------
df["is_in_turn"] = df["Turn"].isin(["contingent", "initiation"])

# 4) prep containers ----------------------------------------------------
classifications = df["Classification"].dropna().unique()
results = []

# 5) loop over all classes ----------------------------------------------
for cls in classifications:
    # split once per iteration
    in_turn = df[df["is_in_turn"]]
    non_turn = df[~df["is_in_turn"]]

    # counts for this class
    k1 = (in_turn["Classification"] == cls).sum()
    k2 = (non_turn["Classification"] == cls).sum()
    n1 = len(in_turn)
    n2 = len(non_turn)

    # Beta(1,1) priors
    post1 = beta(a=1 + k1, b=1 + (n1 - k1))
    post2 = beta(a=1 + k2, b=1 + (n2 - k2))

    # sample
    samples1 = post1.rvs(10_000)
    samples2 = post2.rvs(10_000)
    diff = samples1 - samples2

    mean_diff = diff.mean()
    ci_low, ci_high = np.percentile(diff, [2.5, 97.5])
    prob_gt0 = (diff > 0).mean()

    results.append(
        {
            "classification": cls,
            "mean_diff_inturn_minus_non": round(mean_diff, 3),
            "ci_2.5": round(ci_low, 3),
            "ci_97.5": round(ci_high, 3),
            "p(in_turn > non_turn)": round(prob_gt0, 3),
            "in_turn_count": int(k1),
            "non_turn_count": int(k2),
        }
    )

# 6) to DataFrame and sort ----------------------------------------------
results_df = (
    pd.DataFrame(results)
    .sort_values(by="p(in_turn > non_turn)", ascending=False)
    .reset_index(drop=True)
)

# show only top rows so the notebook/github doesn’t explode
print("\nBayesian comparison of CDS classifications (top 15):")
print(results_df.head(15))

# if you want to save:
# results_df.to_csv("../results/bayesian_turn_comparison.csv", index=False)


### CHARTS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# -------------------------------------------------------
# 1. Input: coefficients + SE from the R hglm output
# -------------------------------------------------------
# R output:
# avg.CDS       -1.065e-02  SE = 5.778e-03  p = 0.0736
# avg.CTC        1.142e-02  SE = 1.094e-03  p = 1.92e-12
# SES            6.735e-02  SE = 6.294e-03  p = 9.90e-13
# avg.CTC:SES   -1.584e-04  SE = 2.013e-05  p = 2.45e-09

data = {
    "label":   ["CDS", "CTC", "SES", "CTC×SES"],
    "coef":    [-1.065e-02,  1.142e-02,  6.735e-02, -1.584e-04],
    "se":      [ 5.778e-03,  1.094e-03,  6.294e-03,  2.013e-05],
    "p":       [ 7.36e-02,   1.92e-12,   9.90e-13,   2.45e-09],
}

df = pd.DataFrame(data)
df["sig"] = df["p"] < 0.05

# 95% CI on the log scale
z = 1.96
df["lower_coef"] = df["coef"] - z * df["se"]
df["upper_coef"] = df["coef"] + z * df["se"]

# convert to IRR
df["irr"] = np.exp(df["coef"])
df["lower_irr"] = np.exp(df["lower_coef"])
df["upper_irr"] = np.exp(df["upper_coef"])

# sort so SES etc. are nicely ordered (top-to-bottom)
df = df.sort_values("irr", ascending=True).reset_index(drop=True)

# -------------------------------------------------------
# 2. Global style
# -------------------------------------------------------
plt.rcParams.update({
    "font.size": 14,
    "axes.titlesize": 18,
    "axes.labelsize": 16,
    "xtick.labelsize": 14,
    "ytick.labelsize": 14,
    "legend.fontsize": 14,
})

# -------------------------------------------------------
# 3. Dot-and-whisker plot (horizontal)
# -------------------------------------------------------
fig, ax = plt.subplots(figsize=(9, 6))

y_pos = np.arange(len(df))

# whiskers
for i, row in df.iterrows():
    ax.hlines(y=i,
              xmin=row["lower_irr"],
              xmax=row["upper_irr"],
              color="gray",
              linewidth=2)

# points
colors = ["steelblue" if sig else "lightgray" for sig in df["sig"]]
ax.scatter(df["irr"], y_pos, color=colors, s=80, zorder=3, edgecolor="black")

# reference line at IRR = 1
ax.axvline(1, color="black", linestyle="--", linewidth=1)

ax.set_yticks(y_pos)
ax.set_yticklabels(df["label"])
ax.set_xlabel("Incident rate ratio (exp(coef))")
ax.set_title("Predictors of 18-month vocabulary (Poisson hGLM)")

# legend for significance
legend_elems = [
    Patch(facecolor="steelblue", edgecolor="black", label="p < .05"),
    Patch(facecolor="lightgray", edgecolor="black", label="n.s.")
]
ax.legend(handles=legend_elems, loc="lower right", frameon=True)

plt.tight_layout()

# -------------------------------------------------------
# 4. Save as PDF (vector)
# -------------------------------------------------------
plt.savefig(r"C:\Users\lvfeu\Desktop\Plot_coefficients.pdf",
            format="pdf", bbox_inches="tight")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np

from matplotlib.font_manager import FontProperties
personal_path = 'C:/Users/lvfeu/Desktop/PhD/University of Zurich/Paper 2/Metaphor Identification Corpus 3 modalities/'

# get normal font and store it in `Helvetica`
font_path = personal_path + 'helveticaneue.ttf'
Helvetica = FontProperties(fname=font_path)

# -----------------------------
# Data Setup
# -----------------------------
ages = ['6 m.o.', '12 m.o. (1)', '12 m.o. (2)', '18 m.o.']
x_positions = [6, 12, 12, 18]
y_positions = [300, 440, 160, 300]  # Stack 12 mo pies vertically
total_dialogues = np.array([125, 515, 501, 336])
child_pct = np.array([0.24, 0.53, 0.499, 0.634])
caregiver_pct = 1 - child_pct
colors = ['#377eb8', '#ff7f00']  # Blue for child, orange for caregiver

# -----------------------------
# Pie Size Scaling
# -----------------------------
max_radius = 2  # control how big the biggest pie is
scaled_radii = np.sqrt(total_dialogues / total_dialogues.max()) * max_radius
pie_diameters = scaled_radii * 2  # convert radius to diameter

# -----------------------------
# Plot Setup
# -----------------------------
fig, ax = plt.subplots(figsize=(15, 10), dpi=600)
ax.set_xlim(5, 20)
ax.set_ylim(0, 600)

ax.set_xlabel("Child's Age (months)", fontsize=24, font=Helvetica, labelpad=15)
ax.set_ylabel('Turn initiation percentage in all dialogues', fontsize=26, font=Helvetica, labelpad=15)
ax.set_title('Dialogue Initiation Across Ages', font=Helvetica, fontsize=30, pad=20)

ax.tick_params(axis='y', which='both', left=False, labelleft=False)
for label in ax.get_xticklabels():
    label.set_fontproperties(Helvetica)
ax.tick_params(axis='x', labelsize=20)

# -----------------------------
# Draw Pies + % Labels
# -----------------------------
for i in range(len(ages)):
    center_x, center_y = x_positions[i], y_positions[i]
    diameter = pie_diameters[i]

    # Pie container
    inset_ax = inset_axes(ax,
                          width=diameter,
                          height=diameter,
                          loc='center',
                          bbox_to_anchor=(center_x, center_y),
                          bbox_transform=ax.transData,
                          borderpad=0)

    # Draw pie
    wedges, _ = inset_ax.pie(
        [child_pct[i], caregiver_pct[i]],
        colors=colors,
        startangle=90,
        wedgeprops=dict(edgecolor='white')
    )
    inset_ax.set_aspect('equal')
    inset_ax.axis('off')

    # Add child % label inside the blue wedge
    inset_ax.text(-0.4, 0.17, f"{round(child_pct[i]*100)}%",
                  color='white', fontsize=25, weight='bold',
                  ha='center', va='center', font=Helvetica)

    # Add age label below each pie
    ax.text(center_x, center_y - diameter * 37, ages[i], ha='center', fontsize=23, font=Helvetica)

# -----------------------------
# Legend
# -----------------------------
legend_labels = ['Child-initiated', 'Caregiver-initiated']
legend_patches = [plt.Line2D([0], [0], marker='o', color='w',
                              markerfacecolor=c, markersize=16)
                  for c in colors]

ax.legend(legend_patches, legend_labels,
          loc='upper right', fontsize=26, frameon=False)

plt.tight_layout()
# Pad margins so that markers don't get clipped by the axes
plt.margins(0.2)

# plt.savefig(r"C:\Users\lvfeu\Desktop\Initiation_Dialogues.jpg", dpi=600, bbox_inches="tight")
plt.savefig(r"C:\Users\lvfeu\Desktop\Plot_coefficients.pdf",
            format="pdf", bbox_inches="tight")
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# -----------------------------
# Data Setup (example from image)
# -----------------------------
data = {
    'Classification': [
        'Description', 'Question', 'Backchannel', 'Imperative', 'Exclamation',
        'Imitation', 'Scaffolding', 'Play', 'Expansion', 'Praise',
        'Iconic', 'Repair', 'Singing'
    ],
    'Within_Dialogue': [9.39, 8.46, 6.39, 3.95, 3.23, 3.27, 2.96, 2.50, 1.82, 1.67, 1.20, 1, 0.45],
    'Outside_Dialogue': [15.74, 10.19, 1.72, 4.32, 5.36, 0.04, 4.94, 5.19, 0.08, 2.11, 1.67, 0.02, 1.45]
}

df = pd.DataFrame(data)
df.sort_values('Outside_Dialogue', ascending=True, inplace=True)
y_pos = np.arange(len(df))

# -----------------------------
# Plot Setup
# -----------------------------
fig, ax = plt.subplots(figsize=(15, 10), dpi=600)

# Draw lines between the two sides
for i in range(len(df)):
    ax.plot(
        [-1 * df['Outside_Dialogue'].iloc[i], 1 * df['Within_Dialogue'].iloc[i]],
        [y_pos[i], y_pos[i]],
        color='gray', lw=2, zorder=1
    )

# Scatter plots (dots)
ax.scatter(-1 * df['Outside_Dialogue'], y_pos, color='#ff7f00', s=200, label='Outside Dialogue', zorder=2)
ax.scatter(1 * df['Within_Dialogue'], y_pos, color='#377eb8', s=200, label='Within Dialogue', zorder=2)

# -----------------------------
# Bold selective labels
# -----------------------------
highlight = {'Imitation', 'Expansion', 'Repair'}
formatted_labels = [
    f"$\\bf{{{label}}}$" if label in highlight else label
    for label in df['Classification']
]
ax.set_yticks(y_pos)
ax.set_yticklabels(formatted_labels, font=Helvetica, fontsize=24)

# -----------------------------
# Axis settings
# -----------------------------
ax.set_xlabel('Proportion (%)', font=Helvetica, fontsize=24)
ax.set_title('Within vs Outside Dialogue: CDS Utterance Classification', font=Helvetica, fontsize=30, pad=15)

ax.axvline(0, color='black', lw=1)
ax.set_xlim(-20, 20)
ax.set_xticks(np.arange(-20, 21, 5))
ax.set_xticklabels([f"{abs(t)}%" for t in np.arange(-20, 21, 5)], font=Helvetica, fontsize=18)

# -----------------------------
# Legend styling
# -----------------------------
legend = ax.legend(loc='lower left')
# Optional: apply custom font
for text in legend.get_texts():
    text.set_fontproperties(Helvetica)
    text.set_fontsize(26)

plt.grid(True, axis='x', linestyle='--', linewidth=0.5, alpha=0.7)
plt.tight_layout()
#plt.margins(0.2)

plt.savefig(r"C:\Users\lvfeu\Desktop\Within_Outside_Dialogues.pdf", dpi=600)
plt.show()


### Reliabilities

In [ ]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from pathlib import Path

# --- 1. load data (relative path) ---
# put the file in e.g. "data/Reliabilities_Test_OlgaS.xlsx"
file_path = Path("data") / "Reliabilities_Test_OlgaS.xlsx"
df = pd.read_excel(file_path, sheet_name="Test")

# --- 2. select the two annotator columns and drop rows where either is missing ---
ann = df[["Classification 1", "Classification 2"]].dropna(subset=["Classification 1", "Classification 2"])

# normalize (strip + lower)
ann = ann.apply(lambda col: col.astype(str).str.strip().str.lower())

# --- 3. Cohen’s kappa ---
kappa = cohen_kappa_score(ann["Classification 1"], ann["Classification 2"])
print(f"Cohen's kappa: {kappa:.2f}")

# --- 4. confusion matrix (to see which labels disagree) ---
labels = sorted(set(ann["Classification 1"]).union(ann["Classification 2"]))
cm = confusion_matrix(ann["Classification 1"], ann["Classification 2"], labels=labels)

fig, ax = plt.subplots(figsize=(6, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(ax=ax, cmap="Blues", xticks_rotation=45, colorbar=False)
ax.set_title("Coder agreement by label")
plt.tight_layout()
plt.show()
